# Time Series Anomaly Exercises

The dataset for these exercises lives in the Codeup Data Science MySQL Server. The database name is curriculum_logs.

Hint: You will need to explore the database and significantly adjust your acquisition step (yay SQL!)

1. Label students by the program they are in.

2. Is it possible to identify the user_id of a staff member?

3. Identify students who are accessing our curriculum pages beyond the end of their time at Codeup.